In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f"""
CREATE OR REPLACE TABLE {catalog_name}.udp_wcm_gold_vcm_dwh.d_mdq_daily 
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
AS
SELECT  store_id,
        product_id,
        mdq,
        external_report_mdq,
        internal_report_mdq,
        old_mdq,
        mdq_age,
        mdq_available,
        replenishment_mode,
        'NON_FRESH' AS sku_type
FROM    {catalog_name}.udp_wcm_gold_vcm_dwh.d_mdq_ambient_daily
WHERE   product_id NOT IN (SELECT product_id FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_mdq_fresh_daily)
UNION ALL
SELECT  store_id,
        product_id,
        mdq,
        mdq AS external_report_mdq,
        mdq AS internal_report_mdq,
        mdq AS old_mdq,
        NULL AS mdq_age,
        mdq_available,
        NULL AS replenishment_mode,
        'FRESH' AS sku_type
FROM    {catalog_name}.udp_wcm_gold_vcm_dwh.d_mdq_fresh_daily;
""")